In [98]:
# !pip install mxnet
# !pip install gluonnlp pandas tqdm
# !pip install sentencepiece
!pip install transformers
# !pip install konlpy
!pip install torch
!pip install ipywidgets  # for vscode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [99]:
#gpu 사용시에만 적용
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Jun 20 07:30:55 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   74C    P0    33W /  70W |  15070MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [100]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [101]:
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm.notebook import tqdm

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [102]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [103]:
data = pd.read_excel('/content/drive/MyDrive/ABP_data.xlsx')

In [104]:
# 2017년 7월 1일 지식제안 체계 개편
from datetime import datetime as dt
data["심의요청일"] = pd.to_datetime(data["심의요청일"], format="%Y/%m/%d")
data["심의완료일"] = pd.to_datetime(data["심의완료일"], format="%Y/%m/%d")
data = data.loc[data["심의요청일"] >= dt(2017, 7, 1)]   

In [105]:
# 지식제안 데이터만 사용
data = data.dropna(subset=['제안구분'])
data = data.loc[data["제안상태"].isin(["불채택", "개발완료", "채택", "개발중"])] # , '심의요청', '심의중'
# 카테고리 3 Depth 만 사용
data = data.loc[data["제안구분길이"] == 3]
# 중복인 경우 최신 데이터만 남기기
data = data.dropna(subset = ['제안명','현황 및 문제점', '개선방안 및 기대효과'])
data = data.sort_values(by=["심의요청일", "현황 및 문제점"])
data = data.drop_duplicates("제안명", keep="last")
data = pd.DataFrame(data)
# 데이터 집계1 - 카테고리별 건수 
print(data.shape)
count_category = pd.DataFrame(data.groupby('심의부서')['제안명'].count())

(20850, 27)


In [106]:
df = pd.DataFrame(data)
df.head(10)

,OID,구분,세부구분,제안번호,키워드,카테고리ID,제안구분,제안명,제안부서,제안자명,...,현황 및 문제점,개선방안 및 기대효과,반송코드,반송사유,기제안번호,심의내용,COLLABORATOR,AI식별자,AI카테고리순위,제안구분길이
37290,3170703145615198754,제안,지식제안,2017-03886,"국민ONE통장,가계부",148.0,수신>수신상품부>요구불예금,국민 ONE통장 입출금거래내역 표시건 표시 수정요청건,효창동지점,박주광,...,,매월 1일에 가계부 기능으로 금액이 표시되는데. 얼핏 보기엔 거래내역이 연속된 ...,반송 11-3,이미 부서에서 검토하였으나 실효성이 낮아 종결한 사항,NaN,바쁘신 와중에 소중한 지식제안 감사드립니다. 가계부기능 통장 인자에 대한 의견...,NaN,NaN,NaN,3
37316,3170703183532057434,제안,지식제안,2017-03914,"태블릿브랜치,신분증 진위확인",437.0,영업지원/기타>아웃바운드지원부>KB태블릿브랜치(시스템운영),태블릿브랜치 신분증 진위확인 처리방법 개선,진천지점,신우관,...,"- 태블릿브랜치에서 신분증 진위확인시 주민등록증은 1382, 운전면허증은 인...",- 창구직원 신분증 진위확인 시스템과 동일하게 신분증 진위확인 버튼 클릭하면 자...,반송 11-7,IT구현 등 기술적 제약 존재,NaN,"태블릿브랜치 업무에 관심을 가져 주셔서 감사합니다. 제안해주신 ""신분증 ...",NaN,NaN,NaN,3
37313,3170703181130026414,제안,지식제안,2017-03912,"자동화기기 미수취,미수취",380.0,영업지원/기타>채널지원부>무인채널 전략수립 (ATM 전략 및 수수료/자동화기기이용번...,자동화기기 미수취 발생시 처리방법 개선자동화기기,진천지점,신우관,...,* 현황 - 자동화기기에서 현금 및 수표 출금시 고객께서 휴대폰 조작...,* 개선방안 - 고객 미수취 발생시 현금 출금시 출금금액을 계수하여 회수하...,반송 11-2,이미 개발 진행 중이거나 시행 중인 것,NaN,귀하의 제안에 감사드립니다.자동화기기의 미수취 장애 발생 시 입금 처리 방법 개...,NaN,NaN,NaN,3
37322,3170703190626760232,제안,지식제안,2017-03920,"자동이체,자동이체 미처리",149.0,수신>수신상품부>자동이체(계좌간/타행/납부자/아파트관리비/선일자),자동이체 미처리시 고객앞 sms발송,동대문패션타운지점,양이지,...,&lt;현황 자동이체 신청건 미이체시 고객에 ' 자동이체가 미처리되었습니다...,&lt;개선방안 및 기대효과 문자발송시 출금계좌번호와 더불어 어떠한건이 미처...,반송 11-2,이미 개발 진행 중이거나 시행 중인 것,NaN,바쁘신 업무중에도 지식제안에 감사드립니다. 현재 자동이체 업무에서 타행자동이체인...,NaN,NaN,NaN,3
37332,3170703212225305192,제안,지식제안,2017-03930,"상담담보평가,면적",251.0,여신>여신코어뱅킹부(P)>여신담보(IT),상담담보평가(11-3F-014) 화면 '우선변제/임금채권' 탭에 면적 입력시 소숫점...,병점중앙지점,이주철,...,&lt;현황 1. 상담담보평가(11-3F-014) 화면의 '우선변제/임금채권...,&lt;개선방안 1. '우선변제/임금채권' 탭에서 면적 입력시 마지막 두 자...,NaN,NaN,NaN,여신담보업무 개선에 관심을 가져주셔서 감사합니다. 다른 화면에서 면적입력...,NaN,NaN,NaN,3
37333,3170703213218821974,제안,지식제안,2017-03931,"면적,우선변제보증금",236.0,여신>기업상품부>담보평가(기업),면적단위 표기,병점중앙지점,이주철,...,&lt;현황 상담담보평가(11-3F-014)의 '우선변제/임금채권' 탭에 면...,&lt;개선방안 면적 입력란 옆(오른쪽)에 '평방미터' 심볼 표기 (참고:...,NaN,NaN,NaN,담보평가 업무에 보내주신 관심과 제안에 감사드립니다. 제안자께서 제안하신 내용...,NaN,NaN,NaN,3
37274,3170703112906540679,제안,지식제안,2017-03868,"담당자,전화번호",409.0,영업지원/기타>혁신추진부>프로세스 개선,마이스타 단말 거래시 해당 업무 화면에서 담당자 조회,죽전역지점,신기욱,...,*현황: 마이스타단말에서 업무시 해당업무의 본부부서 담당자와 통화를 해야 할 ...,*개선방안 :단말 거래시 해당 문서나 장표를 연결해서 볼 수 있는 현 시스템처럼...,반송 11-4,과거 1년간 이미 접수된 제안과 동일,NaN,★반송사유 : 반송 11-3:심의요청일로부터 과거 1년간 이미 접수된 제안과 동...,NaN,NaN,NaN,3
37246,3170703093229941507,제안,지식제안,2017-03836,"외화,매입,수수료,감면",428.0,"외환>외환마케팅부>무역 상품(서비스), 무역 비대면 채널",외화현찰 매입 시 현찰수수료감면구분 전산 개선의 건,포항종합금융센터,권현진,...,- 00-41-002 외국통화매입 전산에서 수수료감면구분 1-본부승인 선택 살행...,1- 수수료 감면 남용 방지 2- 고객 응대 일관성 유지 가능 (A지점에선 감...,반송 11-2,이미 개발 진행 중이거나 시행 중인 것,NaN,"바쁜 업무중에도 외환업무에 관심을 가져주셔서 감사합니다. 말씀하신 거래 시, ...",NaN,NaN,NaN,3
37325,3170703191704233001,제안,지식제안,2017-03924,"기업여신,cif,고객종합정보조회,채권서류,서류편철",203.0,여신>기업상품부>SOHO여신프로세스,"기표 전, 후 고객정보조회(CIF), 신용정보조회, 여신원장조회 일괄조회할 수 있는...",상무지점,김형균,...,"- SOHO 및 법인여신심사 프로세스에서 심사 시점, 그리고 기표 당시 고객의 ...",- 현재 여신승인이 완료되면 보고서리스트를 일괄출력하여 채권철에 편철할 수 있듯...,반송 11-2,이미 개발 진행 중이거나 시행 중인 것,NaN,"바쁘신 중 기업여신프로세스 관련 제안에 감사드리며, 심의가 지연된점 양해말씀 드...",NaN,NaN,NaN,3
37262,3170703104938011821,제안,지식제안,2017-03853,"여신심사,보고서 업무",244.0,여신>여신코어뱅킹부(P)>기업심사승인(IT),소호심사 보고서리스트(04-3B-048)에서 보고서 출력 관련,가경동지점,모경호,...,- 소호심사 보고서리스트(04-3B-048) 화면에서 출력할 보고서를 선택후 출...,- 내가 필요한 보고서를 선택했음에도 출력시 건건이 다시 클릭해서 출력해야 하는...,반송 11-7,IT구현 등 기술적 제약 존재,NaN,기 영업점 요청사항입니다. 그러나 보고서를 출력하는 Tool 에 제약조건이 ...,NaN,NaN,NaN,3


In [107]:
print(df['제안구분'].value_counts())

여신>개인여신부>가계여신일반운용제도                     1294
수신>수신상품부>수신 신규/해지/제신고/제증명                879
수신>수신상품부>요구불예금                           761
영업지원/기타>영업플랫폼부(P)>디지털창구업무                699
퇴직연금>연금기획부>퇴직연금 업무 프로세스 관리 및 개선          682
                                        ... 
카드>고객마케팅부>우수고객 마케팅                         1
영업지원/기타>직원만족부>복지후생(퇴직직원)                   1
영업지원/기타>스마트상담플랫폼부(P)>인바운드(화면개발 및 수정)       1
영업지원/기타>리스크관리부>주요위험지표(KRI)                 1
카드>고객마케팅부>굿세이브서비스                          1
Name: 제안구분, Length: 353, dtype: int64


In [108]:
df.drop(df.loc[df['제안구분']=='수신>수신상품부>계산'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='신탁/ISA>신탁부>신탁대출'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='신탁/ISA>신탁부>특정금전신탁(MMT)'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='여신>여신관리부>대손상각'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='여신>여신코어뱅킹부(P)>여신사후관리(IT)'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>총무부>통신, 인터폰(자동화기기)'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='퇴직연금>연금컨설팅부>퇴직연금 연금계리, 재정검증 등'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='수신>수신상품부>내국환'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='여신>여신코어뱅킹부(P)>여신금리(IT)'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='여신>여신코어뱅킹부(P)>B2B(IT)'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>브랜드전략부>상품광고'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>직원만족부>복지후생(의료비,근무복)'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='퇴직연금>연금기획부(퇴직연금지원Unit)>퇴직연금 후선업무'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='수신>수신코어뱅킹부(P)>공과금/공금예금'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='수신>주택기금부>국민주택채권'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>리스크관리부>리스크자가진단 (CSA)'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>브랜드전략부>KBN위성방송'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>안전관리부>CCTV운영'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>직원만족부>복지후생(장례지원)'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>총무부>시설관리(건축)'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='카드>카드금융부>현금서비스(CA)'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>전략기획부>직제'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>총무부>동산관리(사무/공조기기)'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='퇴직연금>연금컨설팅부>퇴직연금 세무'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='여신>기업경영개선부>기업워크아웃'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='여신>여신관리부>자산건전성 및 연체관리 기획'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='여신>여신코어뱅킹부(P)>기업신용평가(IT)'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='여신>주택기금부>기금사후관리'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>개인마케팅부(마케팅1부)>통장디자인'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>국군마케팅부>군부대/나라사랑카드'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>법률지원부>법무'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>신용리스크부>여신거래한도'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>자금세탁방지부>RBA (자금세탁위험평가)'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>재무기획부>예산'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>전략기획부>전략기획'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>중소기업고객부>기업영업추진'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>총무부>법원문서'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>총무부>시설관리(기계설비)'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>HR부>평가제도개선'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>WM기획부>자산현황통보'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='카드>총무지원부>카드 교부/자재 관리/실물 발행'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='여신>여신코어뱅킹부(P)>이자계산'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>경영정보개발부(P)>경영정보공통'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>기관영업부(P)>지자체/정부'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>리스크관리부>운영리스크 관리'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>인재개발부>연수기획/제도운영,KPI/국내외MBA'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>정보보안부(P)>IT보안(인터넷보안)'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>총무부>동산관리(비품관리)'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>총무부>시설관리(CI/간판)'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>ESG기획부>ESG 및 CSR'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>WM기획부>복합점포 시너지'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>WM투자전략부>사후관리'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='외환>여신코어뱅킹부(P)>전자무역/SWIFT(IT)'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='카드>고객마케팅부>개인회원 통합/Mass 마케팅'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='카드>체크카드사업부>체크카드 마케팅'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='카드>회원영업부>비대면(온라인,모바일) 카드 모집/마케팅'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='수신>수신코어뱅킹부(P)>적립식(IT)'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='여신>여신코어뱅킹부(P)>기한연장'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>스마트상담부>업무상담(4119상담)'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>신용리스크부>산업정보조회시스템'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>아웃바운드지원부>대출상담사'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>영업기획부>영업기획'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>자금운용지원부>지준이체'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>자본시장영업1부>골드통장'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>채널감사부>영업채널감사'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>총무부>자산관리/종합물자'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>HR부>개인평가'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>HR부>인사기획'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='외환>여신코어뱅킹부(P)>수입(IT)'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='외환>외환마케팅부>외국인마케팅'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='카드>고객마케팅부>우수고객 제도 운영 및 관리'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='카드>고객서비스부>고객센터 상담/교육'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='퇴직연금>연금기획부(연금상품운영Unit)>퇴직연금 고객 및 수익률 관리'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='수신>수신코어뱅킹부(P)>가수/가지급, 수입인지/증지'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='여신>데이터분석부(P)>신용정보(IT)'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='여신>여신코어뱅킹부(P)>여신계정등록/변경'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>데이터분석부(P)>민원(IT)'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>스마트상담플랫폼부(P)>인바운드(화면개발 및 수정)'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>인재개발부>경영자예비과정/글로벌인력양성'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>정보보안부(P)>전자금융보안'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>직원만족부>복지후생(건강검진)'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>총무부>구매관리'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>테크기획부>채용'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>DT전략부>혁신서비스'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>HR부>이동 및 공모'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>WM투자자문부>전문가상담예약서비스 기획 및 운영'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='외환>여신코어뱅킹부(P)>외환공통(IT)'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='카드>업무혁신부>체크카드 제도 기획 및 운영'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='퇴직연금>연금기획부>연금사업 기획(사업전략, KPI, 교육/연수 등)'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='수신>수신코어뱅킹부(P)>원천징수, 중요증서'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='여신>기업금융솔루션부(P)(솔루션1부(상품))>공급망금융'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='여신>기업금융솔루션부(P)(솔루션1부(상품))>비대면 기업 대출'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='여신>여신관리부>여신사후관리 SMS,DM'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>경영정보개발부(P)>성과관리(IT)'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>글로벌지원부>해외사업'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>기관영업부(P)>공공기관'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>스마트상담플랫폼부(P)>아웃바운드(화면개발 및 수정)'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>신용리스크부>담보회수율'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>신용리스크부>Total Exposure'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>아웃바운드지원부>집단대출'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>인재개발부>리더십/신임ㆍ승격예비과정/팀빌딩'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>총무부>동산관리(비품배정)'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>총무부>부동산관리'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>총무부>시설관리(365자동화)'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>HR부>겸업/겸직'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>HR부>계약직원 관리'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>HR부>채용'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>HR부>휴직/복직 및 퇴직'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='카드>고객마케팅부>우수고객 마케팅'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='카드>브랜드전략부>광고'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='수신>수신코어뱅킹부(P)>금융공동망'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='수신>수신코어뱅킹부(P)>판매대행'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='신탁/ISA>수신코어뱅킹부(P)>신탁(IT)'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='신탁/ISA>신탁부>부동산맞춤신탁'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='신탁/ISA>신탁부>특정금전신탁(CP)'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='여신>기술금융부>기술평가'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='여신>기업경영개선부>기업회생'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='여신>기업상품부>SOHO 신상품'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='여신>데이터분석부(P)>여신 상품/금리정보'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='여신>여신관리부>신용정보사 채권 회수위임'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='여신>여신관리부>채무조정(가계NPL)'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='여신>여신관리부>채무조정(기업NPL)'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>경영정보개발부(P)>대외기관제공정보등록'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>리스크관리부>주요위험지표(KRI)'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>본점감사부>본부감사'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>신용감리부>신용감리'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>재무기획부>자본금/공시'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>증권대행부>증권대행'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>직원만족부>복지후생(퇴직직원)'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>채널지원부>영업점 디자인 표준메뉴얼'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>테크기획부>IT감사'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>ALM부>자금관리'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>ALM부>자산부채관리'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='외환>여신코어뱅킹부(P)>수출(IT)'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='카드>고객마케팅부>굿세이브서비스'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='카드>소비자보호부>소비자보호/CS(고객만족)'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='카드>업무혁신부>결제계좌/결제일/사용등록'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='카드>영업기획부>대고객 메시지(SMS/이메일/PUSH 등)'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>자금운용지원부>지준이체'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='여신>여신코어뱅킹부(P)>기한연장'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>스마트상담부>업무상담(4119상담)'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='수신>수신코어뱅킹부(P)>적립식(IT)'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='카드>고객마케팅부>개인회원 통합/Mass 마케팅'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>자본시장영업1부>골드통장'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>채널감사부>영업채널감사'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>영업기획부>영업기획'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>신용리스크부>산업정보조회시스템'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='카드>영업기획부>대고객 메시지(SMS/이메일/PUSH 등)'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>자금운용지원부>지준이체'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='여신>여신코어뱅킹부(P)>기한연장'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>스마트상담부>업무상담(4119상담)'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='수신>수신코어뱅킹부(P)>적립식(IT)'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='카드>체크카드사업부>체크카드 마케팅'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='카드>회원영업부>비대면(온라인,모바일) 카드 모집/마케팅'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>KB부동산플랫폼부(P)>아파트시세, 주택통계'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>직원만족부>복지후생(콘도)'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='여신>신용평가모델부>개인신용평가모델(가계용)'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>인재개발부>스터디그룹/상시학습프로그램/자기주도학습비'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='여신>여신관리부>법적절차'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='카드>브랜드전략부>디자인(카드 디자인 등)'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='여신>여신코어뱅킹부(P)>기업심사승인(IT)'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='카드>카드금융부>카드론'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='여신>여신관리부>특수채권'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='여신>주택기금부>기금구입자금'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='수신>수신코어뱅킹부(P)>공통(000과목)'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='여신>여신코어뱅킹부(P)>보증'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='여신>여신코어뱅킹부(P)>여신신규/여신조회'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='카드>체크카드사업부>체크카드 상품 개발 및 관리'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>아웃바운드지원부>Out Bound 마케팅 영업점 코칭 프로그램 운영'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>소비자보호부>고객경험관리'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='여신>여신관리부>개인워크아웃(개인회생/신용회복/개인파산)'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>채널감사부>RAS 전산평가'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='카드>상품기획부>개인범용카드 신상품 개발/제안'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='여신>기업상품부>여신전결'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>HR부>직무사무분담'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>총무부>인장관리'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='수신>수신코어뱅킹부(P)>거치식(IT)'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='여신>여신관리부>개인협약대출,개인보증부대출'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>HR부>상벌'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>회계부>세무'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='고객정보>데이터분석부(P)>고객정보(IT)'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='여신>신용평가모델부>개인신용평가모델(소매형소호)'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='신탁/ISA>신탁부>신탁 신상품'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='외환>FI영업부>해외직접투자/FDI'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>경영정보개발부(P)>사용자권한관리'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='펀드/방카>금융투자상품본부(방카Unit)>방카슈랑스상품'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='여신>스마트상담부(대전상담팀)>단기연체독촉 및 만기안내'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>총무부>동산관리(차량)'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='여신>여신코어뱅킹부(P)>입금상환'].index, inplace=True)
df.drop(df.loc[df['제안구분']=='영업지원/기타>전략기획부>KB금융 시너지 강화방안'].index, inplace=True)

In [109]:
print(df['제안구분'].value_counts())

여신>개인여신부>가계여신일반운용제도                         1294
수신>수신상품부>수신 신규/해지/제신고/제증명                    879
수신>수신상품부>요구불예금                               761
영업지원/기타>영업플랫폼부(P)>디지털창구업무                    699
퇴직연금>연금기획부>퇴직연금 업무 프로세스 관리 및 개선              682
                                            ... 
카드>신용기획부>회원 자격/심사 기준                          10
영업지원/기타>WM기획부>PB(Private Banking)기획 및 운영      10
영업지원/기타>신용리스크부>Sub-Limit                      10
카드>정보보호부>정보보호                                 10
퇴직연금>연금기획부>퇴직연금 비대면(스타뱅킹, 인터넷뱅킹)              10
Name: 제안구분, Length: 190, dtype: int64


In [110]:
#제안명에서 키워드가 분류되므로, 키워드 삭제
#제안구분에는 부서명이 들어가므로, 심의부서만 확인
#총제안 = 제안명, 현황 및 문제점, 개선방안 및 기대효과 취합
df = df[['제안명', '개선방안 및 기대효과', '제안구분']]
df["총제안"] = df["제안명"] + " " + df["개선방안 및 기대효과"]
import re
def extract_word(text):
    hangul = re.compile('[^가-힣]') 
    result = hangul.sub(' ', text) 
    return result

df['총제안'] = df['총제안'].apply(lambda x:extract_word(x))
df = df[['총제안', '제안구분']]
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,총제안,제안구분
37290,국민 통장 입출금거래내역 표시건 표시 수정요청건 매월 일에 가계부 기능으...,수신>수신상품부>요구불예금
37316,태블릿브랜치 신분증 진위확인 처리방법 개선 창구직원 신분증 진위확인 시스템과...,영업지원/기타>아웃바운드지원부>KB태블릿브랜치(시스템운영)
37313,자동화기기 미수취 발생시 처리방법 개선자동화기기 개선방안 고객 미...,영업지원/기타>채널지원부>무인채널 전략수립 (ATM 전략 및 수수료/자동화기기이용번...
37322,자동이체 미처리시 고객앞 발송 개선방안 및 기대효과 문자발송시 ...,수신>수신상품부>자동이체(계좌간/타행/납부자/아파트관리비/선일자)
37332,상담담보평가 화면 우선변제 임금채권 탭에 면적 입력시 소숫점...,여신>여신코어뱅킹부(P)>여신담보(IT)
...,...,...
59279,가계대출상품설명서 서식 개정 고객 자필 기재 내용을 좀 더 진하게 표시하고...,여신>개인여신부>가계여신일반운용제도
59290,원스탑 신규시 전자금융 장애 관련 뱅킹 연동 신규시 통장 미발행 여부 체크를 실...,수신>영업플랫폼부(P)>One-Stop 신규
59291,고객업무처리중 고객의 착오작성시 다시쓰기 기능을 창구직원단말기에도 생성토록 ...,영업지원/기타>영업플랫폼부(P)>디지털창구업무
59304,활용을 통한 근저당설정 이행등록 절차 간소화 개선방안 활용...,여신>개인여신부>담보관리(가계)


In [111]:
df['총제안'] = df['총제안'].str.strip()
df['총제안'] = df['총제안'].map(lambda x: ' '.join(x.split()))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [112]:
# LabelEncoder 함수를 이용해 각 심의부서 number labeling
import numpy as np
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['제안구분'] = le.fit_transform(df['제안구분'])
# df.head(10)
# df_list = df['총제안'].values.tolist()

In [113]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
train, test = train_test_split(df, test_size=0.1, random_state=0)

In [114]:
print(len(train))
print(len(test))

17931
1993


In [115]:
train_texts = train['총제안'].tolist()
test_texts = test['총제안'].tolist()
train_labels = train['제안구분'].tolist()
test_labels = test['제안구분'].tolist()

In [116]:
print(len(train_texts))
print(len(test_texts))
print(len(train_labels))
print(len(test_labels))

17931
1993
17931
1993


In [117]:
print(len(set(train_labels+test_labels)))

190


In [118]:
# train_sent = []
# train_label = []
# test_sent = []
# test_label = []

# for train, test in zip(dataset_train, dataset_test):
#   train_sent.append(train[0])
#   train_label.append(train[1])
#   test_sent.append(test[0])
#   test_label.append(test[1])

# train_df = pd.DataFrame([ x for x in zip(train_sent,train_label)])
# test_df = pd.DataFrame([ x for x in zip(test_sent,test_label)])



class KBALBERTDataset(Dataset):
    def __init__(self, text, labels, tokenizer):
        self.tokenizer = tokenizer
        # self.text = text
        self.labels = labels
        # sentences = self.dataset[0].tolist()
        # inputs = tokenizer(sentences, padding=True, max_length=256)
        inputs = tokenizer(text, padding=True, truncation=True, max_length=256)
        
        self.input_ids = inputs['input_ids']
        self.attention_masks = inputs['attention_mask']
        # self.labels = self.dataset[1].tolist()
        # self.labels = [float(label) for label in self.labels]

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return (self.input_ids[idx], self.attention_masks[idx], self.labels[idx])

In [119]:
def collate_fn(batch):
    input_ids = [item[0] for item in batch]
    attention_masks = [item[1] for item in batch]
    labels = [item[2] for item in batch]
    return torch.LongTensor(input_ids), torch.LongTensor(attention_masks), torch.LongTensor(labels)

In [120]:
# Setting parameters
num_classes = 190
batch_size = 16
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5

In [121]:
from transformers import AlbertModel, AlbertTokenizer, BertTokenizer
from transformers import AutoModel, AutoTokenizer

In [122]:
kb_albert_model_path = "/content/drive/MyDrive/albert-base-char-v3"
tokenizer = AutoTokenizer.from_pretrained(kb_albert_model_path)
model_albert = AutoModel.from_pretrained(kb_albert_model_path)

Some weights of the model checkpoint at /content/drive/MyDrive/albert-base-char-v3 were not used when initializing AlbertModel: ['predictions.dense.weight', 'predictions.bias', 'sop_classifier.classifier.bias', 'predictions.dense.bias', 'predictions.LayerNorm.weight', 'predictions.LayerNorm.bias', 'sop_classifier.classifier.weight', 'predictions.decoder.weight', 'predictions.decoder.bias']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [123]:
train_ds = KBALBERTDataset(train_texts, train_labels, tokenizer)
test_ds = KBALBERTDataset(test_texts, test_labels, tokenizer)

In [124]:
train_dataloader = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, num_workers=2, collate_fn=collate_fn)
test_dataloader = torch.utils.data.DataLoader(test_ds, batch_size=batch_size, num_workers=2, collate_fn=collate_fn)

In [125]:
class KBALBERTClassifier(nn.Module):
    def __init__(self, albert, num_classes=num_classes, hidden_size=768):
        super(KBALBERTClassifier, self).__init__()
        self.albert = albert
        self.classifier = nn.Linear(hidden_size , num_classes)


    def forward(self, input_ids, attention_masks):
        outputs = self.albert(input_ids=input_ids, attention_mask=attention_masks)
        return self.classifier(outputs.pooler_output)

In [126]:
device = torch.device("cuda:0")

In [127]:
#BERT 모델 불러오기
#GPU 사용
model = KBALBERTClassifier(model_albert, num_classes).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [128]:
#상위 5개 정확도 측정을 위한 함수 정의
def calc_accuracy_top_5(X, Y):
  X = X.tolist()
  Y = Y.tolist()

  top_5_indices = []
  indices = []

  #상위 5개 찾기
  for val in X:
    for i in range(5):
      indices.append(val.index(max(val)))
      val.remove(max(val))
    top_5_indices.append(indices)

  #Accuracy 계산
  correct = 0
  for pred, label in zip(top_5_indices, Y):
    if label in pred:
      correct += 1
  
  top_5_acc = correct/len(Y)
  return top_5_acc


In [129]:
from tqdm import tqdm, tqdm_notebook
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (input_ids, attention_masks, labels) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        input_ids = input_ids.to(device)
        attention_masks = attention_masks.to(device)
        labels = labels.to(device)
        out = model(input_ids=input_ids, attention_masks=attention_masks)
        
        loss = loss_fn(out, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy_top_5(out, labels)
        if batch_id % 10 == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (input_ids, attention_masks, labels) in enumerate(tqdm_notebook(test_dataloader)):
        input_ids = input_ids.to(device)
        attention_masks = attention_masks.to(device)
        labels = labels.to(device)
        out = model(input_ids=input_ids, attention_masks=attention_masks)
        test_acc += calc_accuracy_top_5(out, labels)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/1121 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 5.270279884338379 train acc 0.0
epoch 1 batch id 11 loss 5.237624645233154 train acc 0.056818181818181816
epoch 1 batch id 21 loss 5.233063220977783 train acc 0.06845238095238096
epoch 1 batch id 31 loss 5.347616672515869 train acc 0.08064516129032258
epoch 1 batch id 41 loss 5.421058177947998 train acc 0.08536585365853659
epoch 1 batch id 51 loss 5.091184616088867 train acc 0.0784313725490196
epoch 1 batch id 61 loss 5.180326461791992 train acc 0.08094262295081968
epoch 1 batch id 71 loss 5.1845502853393555 train acc 0.0897887323943662
epoch 1 batch id 81 loss 5.10477352142334 train acc 0.09490740740740741
epoch 1 batch id 91 loss 5.27644157409668 train acc 0.09752747252747253
epoch 1 batch id 101 loss 5.04524040222168 train acc 0.10024752475247525
epoch 1 batch id 111 loss 5.124966144561768 train acc 0.11373873873873874
epoch 1 batch id 121 loss 4.990189552307129 train acc 0.1234504132231405
epoch 1 batch id 131 loss 5.175451278686523 train acc 0.1383587786259

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/125 [00:00<?, ?it/s]

...

epoch 9 test acc 0.7908333333333334


  0%|          | 0/1121 [00:00<?, ?it/s]

epoch 10 batch id 1 loss 0.202022522687912 train acc 1.0
epoch 10 batch id 11 loss 0.0877072811126709 train acc 0.9886363636363636
epoch 10 batch id 21 loss 0.6921426057815552 train acc 0.9732142857142857
epoch 10 batch id 31 loss 0.13442771136760712 train acc 0.967741935483871
epoch 10 batch id 41 loss 0.5251641273498535 train acc 0.9664634146341463
epoch 10 batch id 51 loss 0.2984771132469177 train acc 0.9644607843137255
epoch 10 batch id 61 loss 0.34197890758514404 train acc 0.9692622950819673
epoch 10 batch id 71 loss 0.35975420475006104 train acc 0.9674295774647887
epoch 10 batch id 81 loss 0.0337473601102829 train acc 0.9699074074074074
epoch 10 batch id 91 loss 0.32524657249450684 train acc 0.970467032967033
epoch 10 batch id 101 loss 0.10167190432548523 train acc 0.9733910891089109
epoch 10 batch id 111 loss 0.21582099795341492 train acc 0.9740990990990991
epoch 10 batch id 121 loss 0.1346675157546997 train acc 0.9746900826446281
epoch 10 batch id 131 loss 0.34227657318115234 t

  0%|          | 0/125 [00:00<?, ?it/s]

epoch 10 test acc 0.7893333333333333


In [130]:
PATH = 'content/drive/MyDrive/'

torch.save(model, PATH + 'model.pt')  # 전체 모델 저장
torch.save(model.state_dict(), PATH + 'model_state_dict.pt')  # 모델 객체의 state_dict 저장
torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict()
}, PATH + 'all.tar')  # 여러 가지 값 저장, 학습 중 진행 상황 저장을 위해 epoch, loss 값 등 일반 scalar값 저장 가능

FileNotFoundError: ignored